In [1]:
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
import os
import json
import numpy as np

In [12]:
def path_lon_lat_append(year, what, encoding = 'cp949', ):
    
    # 폴더 생성 경로
    fol = './전처리/wi-fi/path/'
    
    wk = pd.read_csv('../전처리/wi-fi/테마형특화단지ap_위경도.csv', encoding='cp949', index_col=['MAC주소'])


    # MAC 코드화 딕셔너리 불러오기
    with open('../mac_code.json', 'r') as f :
        mac_code = json.load(f, encoding='utf-8')

    # key값과 values값 위치 변경
    mac_code = dict(zip(mac_code.values(),mac_code.keys()))

    # 사용되지 않은 Wi-Fi, 목록에서 제거
    for n in wk.index:
        if n not in mac_code.values():
            wk.drop(n, axis=0, inplace=True)

    # 위도로 정렬 (같은 위도 묶기)
    wk.sort_values('위도', inplace=True)

    # 이전 Mac 주소를 담을 변수
    f_mac = None

    # 동일한 위경도를 보유한 Mac 주소 리스트
    repeat = []

    # repeat 리스트를 모은 리스트
    repeat_all = []

    for n in wk.index:
        # 첫행 Skip
        if n == wk.iloc[0].name:
            f_mac = n
            continue
        # 이전 행 위도와 같다면 append
        if wk.loc[f_mac]['위도'] == wk.loc[n]['위도']:
            repeat.append(f_mac)
            repeat.append(n)

        # 전체 리스트 
        else:
            # 빈 리스트는 무시
            if len(repeat) == 0:
                f_mac = n
                continue
            repeat_all.append(list(set(repeat)))
            repeat = []
        f_mac = n

    
    with open('../mac_code.json', 'r') as f :
        mac_code = json.load(f, encoding='utf-8')

    repeat_num_all = []
    repeat_num = []
    for n in repeat_all:
        for i in range(len(n)):
            repeat_num.append(mac_code[n[i]])
        repeat_num_all.append(repeat_num)
        repeat_num = []

    # key값과 values값 위치 변경
    mac_code = dict(zip(mac_code.values(),mac_code.keys()))


    for n in repeat_num_all:
        for i in range(len(n)):
            if i == 0:
                continue
            mac_code[n[i]] = mac_code[n[0]]
    
    
    
    
    count_folder = f'./전처리/김해시_축제/count/{what}/{year}년/'
    Festival_list = os.listdir(count_folder)
    
    
    
    for festi in tqdm(Festival_list):
    # 각 축제 디렉토리 내에서 날짜 디렉토리만 추출
        day_wifi_folder_list = []

        s_path = count_folder + festi +'/'
        
        for file in os.listdir(s_path):
            X = os.path.join(s_path, file)
            if os.path.isdir(X):
                day_wifi_folder_list.append(file)    
        
        
        print(f'{festi} 진행도------------------------------------------------------')
        for d in day_wifi_folder_list:
    
            # csv 파일명 list 생성
            day_wifi_path = s_path + d +'/'
            csv_list = os.listdir(day_wifi_path)
     
            # 병합을 위한 변수
            cnct_df = None

            # csv 파일명으로 변수 생성 
            for n in tqdm(csv_list):
                name = n.split('.')[0]
                df = pd.read_csv(f'{day_wifi_path}{n}', encoding=encoding)

                # 위경도 컬럼 추가 
                df['main_X'] = None
                df['main_Y'] = None

                if what == 'path':
                    df['next_X'] = None
                    df['next_Y'] = None

                for a in range(len(df)):

                    # Main MAC 주소 위경도 추가 (기준 MAC)
                    s_1 = df['MAC_add'][a]
                    df['main_X'][a] = wk.loc[s_1]['경도']
                    df['main_Y'][a] = wk.loc[s_1]['위도']

                    if what == 'path':
                        # 다음 이동경로 MAC 주소 위경도 추가
                        e_1 = df['Next_MAC'][a]
                        df['next_X'][a] = wk.loc[e_1]['경도']
                        df['next_Y'][a] = wk.loc[e_1]['위도']

                # 폴더 생성
                try:
                    directory = f'./전처리/김해시_축제/lon_lat/{what}/{year}년/{festi}/{d}/'
                    if not os.path.exists(directory):
                        os.makedirs(directory)
                except OSError:
                    print ('Error: Creating directory. ' +  directory)

                # 생성한 폴더에 저장
                df.to_csv(f'{directory}/{n}', encoding=encoding, index=False)

                # 데이터프레임 병합
                if n == csv_list:
                    cnct_df = df
                    continue
                cnct_df = pd.concat([cnct_df, df])
            
            
            file_name = csv_list[0].split('_')[:3]
            file_name = '_'.join(file_name)
            # 병합 데이터프레임 저장
            cnct_df.to_csv(f'{directory}/{file_name}_all.csv', encoding=encoding, index=False)

In [13]:
path_lon_lat_append(2022, 'path')

  0%|          | 0/4 [00:00<?, ?it/s]

04월_김해경로위안민속축제 진행도------------------------------------------------------


  0%|          | 0/52 [00:00<?, ?it/s]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[s_1]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[s_1]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[s_1]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[s_1]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

...

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[s_1]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[s_1]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[s_1]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[s_1]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[s_1]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[s_1]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[s_1]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[s_1]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[s_1]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[s_1]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [14]:
path_lon_lat_append(2022, 'stay')

  0%|          | 0/4 [00:00<?, ?it/s]

04월_김해경로위안민속축제 진행도------------------------------------------------------


  0%|          | 0/52 [00:00<?, ?it/s]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[s_1]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[s_1]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[s_1]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[s_1]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

...

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[s_1]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[s_1]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

  0%|          | 0/53 [00:00<?, ?it/s]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[s_1]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[s_1]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[s_1]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[s_1]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[s_1]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[s_1]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16972\168580081.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu